# First steps with PySpark


## Learning objectives

- Get familiar with PySpark RDDs
- Become imbued with the concept of lazyness


Let's start by loading the file from it's URI: `s3://full-stack-bigdata-datasets/Big_Data/tears_in_rain.txt`


In [ ]:
spark
sc = spark.sparkContext

text_rdd = sc.textFile("s3://full-stack-bigdata-datasets/Big_Data/tears_in_rain.txt")

Print out the first line to make sure everything went well.


In [ ]:
text_rdd.take(1)

Out[2]: ["I've seen things you people wouldn't believe. "]

Good you remember how to load a file from a URI, however, most of the time you will need to access files from distributed files systems, like amazon S3, here we will show you how you can access the file from an S3 bucket using specific credentials.


In [ ]:
# When you try to load a non public file you will get an error

text_rdd = sc.textFile(
    "s3://full-stack-bigdata-datasets/Big_Data/tears_in_rain_not_public.txt"
)
text_rdd.take(1)

# Note that you will only get the error after trying to perform an action, that's only then that you are trying to access the data.

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-4165348753901380> in <module> 
 2 
 3 text_rdd = sc . textFile ( "s3://full-stack-bigdata-datasets/Big_Data/tears_in_rain_not_public.txt" ) 
 ----> 4 text_rdd . take ( 1 ) 
 5 
 6 # Note that you will only get the error after trying to perform an action, that's only then that you are trying to access the data. 

 /databricks/spark/python/pyspark/rdd.py in take (self, num) 
 1566 """
 1567 items = [ ] 
 -> 1568 totalParts = self . getNumPartitions ( ) 
 1569 partsScanned = 0 
 1570 

 /databricks/spark/python/pyspark/rdd.py in getNumPartitions (self) 
 467 2 
 468 """
 --> 469 return self . _jrdd . partitions ( ) . size ( ) 
 470 
 471 def filter ( self , f ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 108 def deco ( * a , ** kw ) : 
 109 try : 
 --> 110 return f ( * a , ** kw ) 
 111 except py4j . protocol . Py4JJavaError as e : 
 112 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o316.partitions.
: java.nio.file.AccessDeniedException: s3://full-stack-bigdata-datasets/Big_Data/tears_in_rain_not_public.txt: getFileStatus on s3://full-stack-bigdata-datasets/Big_Data/tears_in_rain_not_public.txt: com.amazonaws.services.s3.model.AmazonS3Exception: Forbidden; request: HEAD https://full-stack-bigdata-datasets.s3.eu-west-3.amazonaws.com Big_Data/tears_in_rain_not_public.txt {} Hadoop 2.7.4, aws-sdk-java/1.11.655 Linux/5.4.0-1039-aws OpenJDK_64-Bit_Server_VM/25.282-b08 java/1.8.0_282 scala/2.12.10 vendor/Azul_Systems,_Inc. com.amazonaws.services.s3.model.GetObjectMetadataRequest; Request ID: 2B19GCQY0A6EVMBF, Extended Request ID: zzjmYTQrwdPCRzLKLYidAGG4jlapODg3u/qzgDuQ7X2b6EPxHNh6ura+8VPaBApEPcgg7U3npzw=, Cloud Provider: AWS, Instance ID: i-0d7c7f70557ae6b5a (Service: Amazon S3; Status Code: 403; Error Code: 403 Forbidden; Request ID: 2B19GCQY0A6EVMBF; S3 Extended Request ID: zzjmYTQrwdPCRzLKLYidAGG4jlapODg3u/qzgDuQ7X2b6EPxHNh6ura+8VPaBApEPcgg7U3npzw=), S3 Extended Request ID: zzjmYTQrwdPCRzLKLYidAGG4jlapODg3u/qzgDuQ7X2b6EPxHNh6ura+8VPaBApEPcgg7U3npzw=:403 Forbidden
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:243)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:154)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:2854)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:2824)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:2763)
	at org.apache.hadoop.fs.Globber.getFileStatus(Globber.java:57)
	at org.apache.hadoop.fs.Globber.glob(Globber.java:252)
	at org.apache.hadoop.fs.FileSystem.globStatus(FileSystem.java:1676)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.globStatus(S3AFileSystem.java:3662)
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:259)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:213)
	at org.apache.sp

1. Load the file from `filepath` into a PySpark RDD into a `text_file` variable.


2. Print out `text_file`.


Out[3]: s3://full-stack-bigdata-datasets/Big_Data/tears_in_rain_not_public.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

3. That doesn't tell us much, what would you do to see the first 3 elements of this RDD? Print out the first 3 elements of the `text_file`.


Out[4]: ["I've seen things you people wouldn't believe. ",
 'Attack ships on fire off the shoulder of Orion. ',
 'I watched C-beams glitter in the dark near the Tannhäuser Gate. ']

4. What's the type of `text_file`?


Out[5]: pyspark.rdd.RDD

It's a PySpark `RDD`. It means we can call **actions** on it and it will return a result.

We want the results to be all elements of the `rdd`.


5. collect all elements of `text_file`.


Out[6]: ["I've seen things you people wouldn't believe. ",
 'Attack ships on fire off the shoulder of Orion. ',
 'I watched C-beams glitter in the dark near the Tannhäuser Gate. ',
 'All those moments will be lost in time, like tears in rain. ',
 'Time to die.']

6. How many lines are there in `text_file`? Count the number of lines in `text_file`.


Out[7]: 5

7. What's the length of each sentence? Call `.map(...)` on your rdd and give it a function that computes the lenght of a string: `lineLengths`

_NOTE: `lineLengths` is how you should name your result variable_


8. Take the first 3 elements of lineLengths


Out[10]: [46, 48, 64]

9. Collect all elements of lineLenghts


Out[12]: [46, 48, 64, 60, 12]

10. What's the average length? Compute the average value of `lineLengths`: `avgLength`


11. What's the type of `avgLength`? Print it out.


Out[12]: float

12. Print out `avgLength`


Out[13]: 46.0

13. Now we want to compute the total length of the document. Compute the sum of all `lineLengths`: `totalLength`


14. What's the type of `totalLength`


Out[15]: int

15. Print out `totalLength`


Out[16]: 230

## Bonus: another way to compute the sum would be to use a `reducer`

This is a step exercise to get you prepare for the next (optional) assignment.

Your goal is to compute the sum of lineLenghts, just like we did, but this time using `.reduce(...)`.  
Here is the link to the [documentation](https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions).


16. Try to compute the total sum, but this time using `.reduce(...)`


Out[24]: 230